In [1]:
import pandas as pd

# Dosya adını kendi dosya yoluna göre ayarla
df = pd.read_csv("StyleApapterDataset.csv")

print("İlk 5 satır:")
print(df.head())

print("\nSınıf dağılımı:")
print(df["label"].value_counts())


İlk 5 satır:
                                                text     label
0  olum neden siyahi herkese kaç santim diye soru...  informal
1  her biri aşagilik birer militansoytari yurtdis...  informal
2  bayadır tweet atmıyormuşum arkadaşlar mutluyum...  informal
3  YÖNLENDİR Birleşmiş Milletler Endüstriyel Geli...    formal
4  Zaten genelde ben de ancak bir kez görebiliyor...   neutral

Sınıf dağılımı:
label
informal    1000
formal      1000
neutral     1000
Name: count, dtype: int64


In [2]:
from sklearn.model_selection import train_test_split

X = df["text"]
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,        # %80 train – %20 test
    random_state=42,
    stratify=y            # sınıf dengesi korunsun
)

print("Train boyutu:", len(X_train))
print("Test  boyutu:", len(X_test))


Train boyutu: 2400
Test  boyutu: 600


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    max_features=5000,     # en çok kullanılan 5000 özellik
    ngram_range=(1, 2),    # unigram + bigram
    stop_words=None        # Türkçe stopword eklemedik, gerekirse sonradan ekleriz
)

X_train_vec = tfidf.fit_transform(X_train)
X_test_vec  = tfidf.transform(X_test)

print("TF-IDF train shape:", X_train_vec.shape)
print("TF-IDF test  shape:", X_test_vec.shape)


TF-IDF train shape: (2400, 5000)
TF-IDF test  shape: (600, 5000)


In [4]:
from sklearn.linear_model import LogisticRegression

lr_style = LogisticRegression(
    max_iter=2000,
    C=2.0,
    n_jobs=-1
)

lr_style.fit(X_train_vec, y_train)
print("Model eğitimi bitti.")


Model eğitimi bitti.


In [5]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = lr_style.predict(X_test_vec)

print("=== Classification Report ===")
print(classification_report(y_test, y_pred))

print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, y_pred, labels=["formal", "informal", "neutral"]))


=== Classification Report ===
              precision    recall  f1-score   support

      formal       0.84      0.86      0.85       200
    informal       0.73      0.75      0.74       200
     neutral       0.70      0.66      0.68       200

    accuracy                           0.76       600
   macro avg       0.75      0.76      0.76       600
weighted avg       0.75      0.76      0.76       600

=== Confusion Matrix ===
[[173   7  20]
 [ 13 150  37]
 [ 21  48 131]]


In [16]:
import random

print("RANDOM TEST SAMPLES".center(60, "-"))

for i in range(10):
    idx = random.randint(0, len(X_test) - 1)
    text = X_test.iloc[idx]
    actual = y_test.iloc[idx]

    pred_proba = lr_style.predict_proba(tfidf.transform([text]))[0]
    pred_label = lr_style.predict(tfidf.transform([text]))[0]

    # Tahmin edilen sınıfın güven skorunu al
    label_index = list(lr_style.classes_).index(pred_label)
    confidence = float(pred_proba[label_index])

    print(f"\n📝 Text: {text}")
    print(f"🎯 Actual Label:    {actual}")
    print(f"🤖 Predicted Label: {pred_label}")
    print(f"🔎 Confidence:      {confidence:.2f}")
    print("-" * 60)


--------------------RANDOM TEST SAMPLES---------------------

 Text: Declanı kaçırmak Gerace ile ona başka ne kazandırır ki?
 Actual Label:    neutral
 Predicted Label: neutral
 Confidence:      0.65
------------------------------------------------------------

 Text: Sanki portakal dünyası; bu otomattan portakal suyu siparişi verince, sadece taze sıkmıyor aynı zamanda içeceğiniz bardağı da portakal kabuklarından üretiyor... Yani hem içecek hem de bardak doğadan.
 Actual Label:    informal
 Predicted Label: informal
 Confidence:      0.56
------------------------------------------------------------

 Text: cool selfieleriyle instagramı coşturan arkadaşım ve herhangi bir selfiede ciddi kalamayan ben
 Actual Label:    informal
 Predicted Label: neutral
 Confidence:      0.63
------------------------------------------------------------

 Text: Arkadaşı ve yönetmen Blake Edwards ile gergin bir ilişkisi vardı.
 Actual Label:    formal
 Predicted Label: formal
 Confidence:      0.70
--------

In [17]:
def predict_style(text: str):
    vec = tfidf.transform([text])
    proba = lr_style.predict_proba(vec)[0]
    label = lr_style.predict(vec)[0]

    label_index = list(lr_style.classes_).index(label)
    confidence = float(proba[label_index])

    return label, confidence

examples = [
    "Sayın yetkili, ilettiğiniz bilgi için teşekkür ederim.",
    "Kanka yarın ne yapıyoruz, netleştirelim artık.",
    "Yarın saat 3'te toplantı yapılacaktır.",
    "Lelelelelele?",
    "Hönkürtme dayı yav"
]

for t in examples:
    label, conf = predict_style(t)
    print(f"\n Text: {t}")
    print(f" Style: {label}  (conf={conf:.2f})")



 Text: Sayın yetkili, ilettiğiniz bilgi için teşekkür ederim.
 Style: neutral  (conf=0.63)

 Text: Kanka yarın ne yapıyoruz, netleştirelim artık.
 Style: informal  (conf=0.69)

 Text: Yarın saat 3'te toplantı yapılacaktır.
 Style: neutral  (conf=0.47)

 Text: Lelelelelele?
 Style: neutral  (conf=0.39)

 Text: Hönkürtme dayı yav
 Style: neutral  (conf=0.39)


In [14]:
import pickle

with open("style_model.pkl", "wb") as f:
    pickle.dump(lr_style, f)

with open("style_vectorizer.pkl", "wb") as f:
    pickle.dump(tfidf, f)

print("✅ style_model.pkl ve style_vectorizer.pkl kaydedildi.")


✅ style_model.pkl ve style_vectorizer.pkl kaydedildi.
